In [1]:
import os
import numpy as np
import librosa
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import joblib

# ───── PATH CONFIG ─────
MEMBO_PATH = "D:/FYP/Datasets/Github Sourced/DroneAudioDataset-master/DroneAudioDataset-master/Multiclass_Drone_Audio/membo_1"
BEBOP_PATH = "D:/FYP/Datasets/Github Sourced/DroneAudioDataset-master/DroneAudioDataset-master/Multiclass_Drone_Audio/bebop_1"
NOISE_PATH = "D:/FYP/Datasets/Github Sourced/DroneAudioDataset-master/DroneAudioDataset-master/Multiclass_Drone_Audio/unknown"
DRONEA_PATH = "D:/FYP/Final Project/SegmentedAudio"
# ───────────────────────

# ───── SETTINGS ─────
SAMPLE_RATE = 16000
N_MFCC = 13
MAX_FRAMES = 40
# ───────────────────

def load_and_label(folder, label):
    data = []
    for fname in os.listdir(folder):
        if fname.endswith('.wav'):
            path = os.path.join(folder, fname)
            y, _ = librosa.load(path, sr=SAMPLE_RATE)
            data.append((y, label))
    return data

def extract_mfcc(signal):
    mfcc = librosa.feature.mfcc(y=signal, sr=SAMPLE_RATE, n_mfcc=N_MFCC)
    if mfcc.shape[1] < MAX_FRAMES:
        pad = MAX_FRAMES - mfcc.shape[1]
        mfcc = np.pad(mfcc, ((0, 0), (0, pad)), mode='constant')
    else:
        mfcc = mfcc[:, :MAX_FRAMES]
    return np.expand_dims(mfcc, axis=-1)  # shape: (13, 40, 1)

# 1. Load and label
membo_data = load_and_label(MEMBO_PATH, 'quadcopter')
bebop_data = load_and_label(BEBOP_PATH, 'quadcopter')
noise_data = load_and_label(NOISE_PATH, 'noise')

train_data = membo_data + bebop_data + noise_data
random.shuffle(train_data)

# 2. Extract features
X = np.array([extract_mfcc(sig) for sig, _ in train_data])
y = [label for _, label in train_data]

# 3. Encode labels
le = LabelEncoder()
y_encoded = to_categorical(le.fit_transform(y))

# Save label encoder
joblib.dump(le, 'label_encoder_quadcopter.pkl')

# 4. Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# 5. Build CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(13, 40, 1)),
    MaxPooling2D((2, 2)),
    Dropout(0.3),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.3),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(y_train.shape[1], activation='softmax')
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# 6. Train
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))
model.save('cnn_quadcopter_model.h5')


d:\FYP\Final Project\myenv\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=24
  warnings.warn(
d:\FYP\Final Project\myenv\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1266
  warnings.warn(


Epoch 1/20


d:\FYP\Final Project\myenv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


293/293 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.8747 - loss: 2.4537 - val_accuracy: 0.9757 - val_loss: 0.0794
Epoch 2/20
293/293 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9630 - loss: 0.1133 - val_accuracy: 0.9744 - val_loss: 0.1470
Epoch 3/20
293/293 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9754 - loss: 0.0863 - val_accuracy: 0.9889 - val_loss: 0.0264
Epoch 4/20
293/293 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9863 - loss: 0.0476 - val_accuracy: 0.9927 - val_loss: 0.0212
Epoch 5/20
293/293 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9896 - loss: 0.0354 - val_accuracy: 0.9940 - val_loss: 0.0195
Epoch 6/20
293/293 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9888 - loss: 0.0338 - val_accuracy: 0.9966 - val_loss: 0.0177
Epoch 7/20
293/293 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9946 - loss: 0.0199 - val_accuracy: 0.9940 - val_loss: 0.0286
Epoch 8/20
293/293 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9918 - loss: 0.0265 - val_accuracy: 0.9983 - val_

In [2]:
import os
import numpy as np
import librosa
import joblib
from tensorflow.keras.models import load_model

# ── CONFIG ─────────────────────
DRONEA_PATH = "D:/FYP/Final Project/SegmentedAudio"  # Folder with DroneA .wav segments
MODEL_PATH = 'D:\FYP\Final Project\cnn_quadcopter_model.h5'
ENCODER_PATH = 'label_encoder_quadcopter.pkl'
SAMPLE_RATE = 16000
N_MFCC = 13
MAX_FRAMES = 40
# ──────────────────────────────

# Load model and label encoder
model = load_model(MODEL_PATH)
label_encoder = joblib.load(ENCODER_PATH)
target_class = 'quadcopter'

def extract_mfcc(signal):
    mfcc = librosa.feature.mfcc(y=signal, sr=SAMPLE_RATE, n_mfcc=N_MFCC)
    if mfcc.shape[1] < MAX_FRAMES:
        pad = MAX_FRAMES - mfcc.shape[1]
        mfcc = np.pad(mfcc, ((0, 0), (0, pad)), mode='constant')
    else:
        mfcc = mfcc[:, :MAX_FRAMES]
    return np.expand_dims(mfcc, axis=-1)  # (13, 40, 1)

# Predict on DroneA files
count_total = 0
count_quadcopter = 0

for fname in os.listdir(DRONEA_PATH):
    if fname.endswith('.wav'):
        path = os.path.join(DRONEA_PATH, fname)
        y, _ = librosa.load(path, sr=SAMPLE_RATE)
        mfcc = extract_mfcc(y)
        X = mfcc[np.newaxis, :, :, np.newaxis]

        probs = model.predict(X, verbose=0)[0]
        predicted_label = label_encoder.inverse_transform([np.argmax(probs)])[0]

        count_total += 1
        if predicted_label == target_class:
            count_quadcopter += 1

print(f"\n🔍 Out of {count_total} DroneA files:")
print(f"✅ {count_quadcopter} predicted as 'quadcopter'")
print(f"❌ {count_total - count_quadcopter} predicted as 'noise'")


<>:9: SyntaxWarning: invalid escape sequence '\F'
<>:9: SyntaxWarning: invalid escape sequence '\F'
C:\Users\Muhammad Husnain\AppData\Local\Temp\ipykernel_3516\3099890160.py:9: SyntaxWarning: invalid escape sequence '\F'
  MODEL_PATH = 'D:\FYP\Final Project\cnn_quadcopter_model.h5'



🔍 Out of 330 DroneA files:
✅ 0 predicted as 'quadcopter'
❌ 330 predicted as 'noise'
